In [1]:
import requests
import json
import os
import pandas as pd
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from apiclient.http import MediaFileUpload

In [2]:
api_key = "DEMO_KEY"
date = "2016-02-02"
request_url = "https://api.nasa.gov/mars-photos/api/v1/rovers/curiosity/photos?api_key=" + api_key + "&earth_date=" + date
response = requests.get(request_url)

In [3]:
cwd = os.getcwd()
# import all image data into a DataFrame for exploration and easy CSV export
all_images = json.loads(response.content)['photos']
df = pd.read_json(json.dumps(all_images))
df.to_csv(cwd + '/metadata.csv', index = False)
df.head()

,camera,earth_date,id,img_src,rover,sol
0,"{'id': 20, 'name': 'FHAZ', 'rover_id': 5, 'ful...",2016-02-02,536759,http://mars.jpl.nasa.gov/msl-raw-images/proj/m...,"{'id': 5, 'name': 'Curiosity', 'landing_date':...",1241
1,"{'id': 20, 'name': 'FHAZ', 'rover_id': 5, 'ful...",2016-02-02,536760,http://mars.jpl.nasa.gov/msl-raw-images/proj/m...,"{'id': 5, 'name': 'Curiosity', 'landing_date':...",1241
2,"{'id': 20, 'name': 'FHAZ', 'rover_id': 5, 'ful...",2016-02-02,536792,http://mars.jpl.nasa.gov/msl-raw-images/proj/m...,"{'id': 5, 'name': 'Curiosity', 'landing_date':...",1241
3,"{'id': 20, 'name': 'FHAZ', 'rover_id': 5, 'ful...",2016-02-02,536793,http://mars.jpl.nasa.gov/msl-raw-images/proj/m...,"{'id': 5, 'name': 'Curiosity', 'landing_date':...",1241
4,"{'id': 20, 'name': 'FHAZ', 'rover_id': 5, 'ful...",2016-02-02,536794,http://mars.jpl.nasa.gov/msl-raw-images/proj/m...,"{'id': 5, 'name': 'Curiosity', 'landing_date':...",1241


In [4]:
# Set up Drive API v3 access
# The following scope gives complete upload/download/delete access
SCOPES = ['https://www.googleapis.com/auth/drive'] 
creds = None

# Once authenticated, file 'token.pickle' will store access codes so that 
# login does not need to occur again
# Code sourced from Drive API's Quickstart sample: https://developers.google.com/people/quickstart/python
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
        
# If file does not have valid creds, prompt user login
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)

    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

service = build('drive', 'v3', credentials=creds)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=15383142439-va1dj0r9bbiba4dfbvsemddup2el10m5.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A52190%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=tsOLN4WAc75IMRnFD5nc2ANnm00Ebd&access_type=offline


In [6]:
# upload csv to Drive
file_metadata = {'name': "metadata.csv", 'parents': ['1zs1NkhuqKLHuaCpdZGwvMyXh79Xi-AsG']}
media = MediaFileUpload("metadata.csv")
file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()

In [5]:
# try to make an "images" directory if it doesn't already exist
try:
    os.mkdir("images")

except:
    print("Directory 'images' already exists.")

Directory 'images' already exists.


In [ ]:
# download image one by one and upload to drive
# even though all links are currently working, added try-except in case a link goes down 
# in the future
for image in all_images:
    try:
        file = requests.get(image['img_src'])
        file_name = str(image['id'])
        ext = "." + image['img_src'].split('.')[-1]
        open(cwd + '/images/' + file_name + ext, 'wb').write(file.content)
        file_metadata = {'name': file_name + ext, 'parents': ['1zs1NkhuqKLHuaCpdZGwvMyXh79Xi-AsG']}
        media = MediaFileUpload('images/'+ file_name + ext,
                                mimetype='image/jpeg')
        file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    
    except:
        print("Image ID:", str(image['id']), "at:", str(image['img_src']), "not found.")


Image ID: 536759 at: http://mars.jpl.nasa.gov/msl-raw-images/proj/msl/redops/ods/surface/sol/01241/opgs/edr/fcam/FLB_507659589EDR_F0521162FHAZ00323M_.JPG not found.
Image ID: 536760 at: http://mars.jpl.nasa.gov/msl-raw-images/proj/msl/redops/ods/surface/sol/01241/opgs/edr/fcam/FRB_507659589EDR_F0521162FHAZ00323M_.JPG not found.
Image ID: 536792 at: http://mars.jpl.nasa.gov/msl-raw-images/proj/msl/redops/ods/surface/sol/01241/opgs/edr/fcam/FRB_507675265EDR_F0521162FHAZ00206M_.JPG not found.
Image ID: 536793 at: http://mars.jpl.nasa.gov/msl-raw-images/proj/msl/redops/ods/surface/sol/01241/opgs/edr/fcam/FLB_507675265EDR_F0521162FHAZ00206M_.JPG not found.
Image ID: 536794 at: http://mars.jpl.nasa.gov/msl-raw-images/proj/msl/redops/ods/surface/sol/01241/opgs/edr/fcam/FLB_507661330EDR_F0521162FHAZ00206M_.JPG not found.
Image ID: 536795 at: http://mars.jpl.nasa.gov/msl-raw-images/proj/msl/redops/ods/surface/sol/01241/opgs/edr/fcam/FRB_507661330EDR_F0521162FHAZ00206M_.JPG not found.
Image ID: 

Image ID: 537717 at: http://mars.jpl.nasa.gov/msl-raw-images/proj/msl/redops/ods/surface/sol/01241/soas/rdr/ccam/CR0_507669064PRC_F0521162CCAM01241L3.PNG not found.
Image ID: 537247 at: http://mars.jpl.nasa.gov/msl-raw-images/msss/01241/mhli/1241MH0005710020403641C00_DXXX.jpg not found.
Image ID: 537248 at: http://mars.jpl.nasa.gov/msl-raw-images/msss/01241/mhli/1241MH0005710010403640C00_DXXX.jpg not found.
Image ID: 537249 at: http://mars.jpl.nasa.gov/msl-raw-images/msss/01241/mhli/1241MH0005710020403630C00_DXXX.jpg not found.
Image ID: 537250 at: http://mars.jpl.nasa.gov/msl-raw-images/msss/01241/mhli/1241MH0005710010403629C00_DXXX.jpg not found.
Image ID: 537251 at: http://mars.jpl.nasa.gov/msl-raw-images/msss/01241/mhli/1241MH0005550020403627C00_DXXX.jpg not found.
Image ID: 537252 at: http://mars.jpl.nasa.gov/msl-raw-images/msss/01241/mhli/1241MH0005550010403626C00_DXXX.jpg not found.
Image ID: 537253 at: http://mars.jpl.nasa.gov/msl-raw-images/msss/01241/mhli/1241MH00057000204036